In [ ]:
from transformers import AutoTokenizer
# model_path = "/workspaces/disk0/data/llm_weights/Platypus2-70B-instruct/"
# model_path = "/workspaces/disk0/data/llm_weights/vicuna-13b-v1.5/"
# model_path = "/data/llm_weights/gptq/MingAI-70B-chat-orca_v0.42_2_dpo-GPTQ/"
model_path = "/data/llm_weights/gptq/CodeLlama-70B-Instruct-GPTQ/"
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    model_max_length=16384,
    padding_side="right",
    use_fast=False,
)
tokenizer.pad_token = tokenizer.unk_token
from transformers.trainer_pt_utils import LabelSmoother
import torch
IGNORE_TOKEN_ID = LabelSmoother.ignore_index

In [ ]:
def random_select(dataset, num_select):
    return dataset.select(np.random.choice(list(range(len(dataset))), num_select))

In [ ]:
dataset1 = load_dpo_dataset("/data/llm_datasets/custom/ados/yarn/yarn-train-tokenized-16k-mistral-200.json")
dataset2 = load_dpo_dataset("/data/llm_datasets/custom/ados/yarn/ko-yarn-mistral-16k-gpt4_evol.json")
dataset3 = load_dpo_dataset("/data/llm_datasets/custom/ados/yarn/ko-yarn-mistral-16k-sharegpt_V3_format_ko.json")
dataset3 = random_select(dataset3, 1000 - len(dataset1) - len(dataset2))

In [ ]:
from datasets import Features, Sequence, Value

dataset2 = dataset2.cast(
    Features({
        'attention_mask': Sequence(Value('int64')),
        'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
        'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
    })
)
dataset3 = dataset3.cast(
    Features({
        'attention_mask': Sequence(Value('int64')),
        'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
        'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
    })
)

In [ ]:
conbined_dataset = concatenate_datasets((dataset1, dataset2, dataset3))

In [ ]:
conbined_dataset = conbined_dataset.shuffle()

In [ ]:
new_dataset = []
for _data in conbined_dataset:
    new_dataset.append(_data)
    
with open('/data/llm_datasets/custom/ados/yarn/ko-yarn-v1.json', "w") as json_file:
    json.dump(new_dataset, json_file)